In [8]:
# OpenAI API Examples Notebook
# 
# This notebook demonstrates key features of the OpenAI API for developers.
# It provides practical examples of the most important capabilities with executable code.

# First, let's set up our environment by installing the OpenAI Python library
%pip install openai numpy matplotlib pydantic --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import the OpenAI library
import os
from openai import OpenAI

In [3]:
MODEL = "gpt-4.1-mini"

In [ ]:
# Initialize the client
# Note: In a real application, you would use an environment variable or secure method
# to store your API key. This is just for demonstration.
client = OpenAI(
    # Replace with your actual API key or use: api_key=os.environ.get("OPENAI_API_KEY")
    api_key="YOUR_API_KEY_HERE"
)

## 1. Basic Text Generation

In [5]:
prompt = "Write a one-sentence bedtime story about a unicorn."

response = client.responses.create(
    model=MODEL,
    input=prompt
)

print(f"Prompt: {prompt}")
print(f"Response: {response.output_text}")

# Show how to control generation with parameters
print("\nWith controlled parameters:")
response = client.responses.create(
    model=MODEL,
    input=prompt,
    temperature=0.7,  # Lower for more deterministic outputs
    top_p=0.9         # Nucleus sampling
)

print(f"Response (controlled): {response.output_text}")

Prompt: Write a one-sentence bedtime story about a unicorn.
Response: Under a sky sprinkled with twinkling stars, a gentle unicorn with a shimmering silver mane whispered magical dreams to the sleeping forest.

With controlled parameters:
Response (controlled): Under a sky sprinkled with twinkling stars, a gentle unicorn with a shimmering silver horn whispered magical dreams to every child as they drifted peacefully to sleep.


---

## 2. Structured Outputs

In [6]:
import json

response = client.responses.create(
   model=MODEL,
    input=[
        {"role": "system", "content": "You are a UI generator AI. Convert the user input into a UI."},
        {"role": "user", "content": "Make a User Profile Form"}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "ui",
            "description": "Dynamically generated UI",
            "schema": {
                "type": "object",
                "properties": {
                    "type": {
                        "type": "string",
                        "description": "The type of the UI component",
                        "enum": ["div", "button", "header", "section", "field", "form"]
                    },
                    "label": {
                        "type": "string",
                        "description": "The label of the UI component, used for buttons or form fields"
                    },
                    "children": {
                        "type": "array",
                        "description": "Nested UI components",
                        "items": {"$ref": "#"}
                    },
                    "attributes": {
                        "type": "array",
                        "description": "Arbitrary attributes for the UI component, suitable for any element",
                        "items": {
                            "type": "object",
                            "properties": {
                              "name": {
                                  "type": "string",
                                  "description": "The name of the attribute, for example onClick or className"
                              },
                              "value": {
                                  "type": "string",
                                  "description": "The value of the attribute"
                              }
                          },
                          "required": ["name", "value"],
                          "additionalProperties": False
                      }
                    }
                },
                "required": ["type", "label", "children", "attributes"],
                "additionalProperties": False
            },
            "strict": True,
        },
    },
)

ui = json.loads(response.output_text)


In [7]:
ui

{'type': 'form',
 'label': 'User Profile Form',
 'children': [{'type': 'field',
   'label': 'First Name',
   'children': [],
   'attributes': [{'name': 'type', 'value': 'text'},
    {'name': 'name', 'value': 'firstName'}]},
  {'type': 'field',
   'label': 'Last Name',
   'children': [],
   'attributes': [{'name': 'type', 'value': 'text'},
    {'name': 'name', 'value': 'lastName'}]},
  {'type': 'field',
   'label': 'Email',
   'children': [],
   'attributes': [{'name': 'type', 'value': 'email'},
    {'name': 'name', 'value': 'email'}]},
  {'type': 'field',
   'label': 'Password',
   'children': [],
   'attributes': [{'name': 'type', 'value': 'password'},
    {'name': 'name', 'value': 'password'}]},
  {'type': 'field',
   'label': 'Date of Birth',
   'children': [],
   'attributes': [{'name': 'type', 'value': 'date'},
    {'name': 'name', 'value': 'dob'}]},
  {'type': 'field',
   'label': 'Bio',
   'children': [],
   'attributes': [{'name': 'type', 'value': 'textarea'},
    {'name': 'nam

---

## 2.1 Structured Outputs with Pydantic models

In [17]:
from pydantic import BaseModel

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message

# If the model refuses to respond, you will get a refusal message
if (math_reasoning.refusal):
    print(math_reasoning.refusal)
else:
    print(math_reasoning.parsed)

steps=[Step(explanation='Start with the equation 8x + 7 = -23. First, isolate the term with x by subtracting 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7; simplifying gives 8x = -30.'), Step(explanation='Now, solve for x by dividing both sides of the equation by 8.', output='(8x)/8 = (-30)/8; simplifying gives x = -30/8.'), Step(explanation='Simplify the fraction -30/8 by dividing numerator and denominator by their greatest common divisor, which is 2.', output='x = (-30 ÷ 2)/(8 ÷ 2) = -15/4.')] final_answer='x = -15/4'


In [22]:
math_reasoning.parsed.steps[0]

Step(explanation='Start with the equation 8x + 7 = -23. First, isolate the term with x by subtracting 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7; simplifying gives 8x = -30.')

---

## 3. Multimodal Capabilities - Vision

In [23]:
# For notebook demonstration, we'll use a placeholder URL
image_url = "https://images.unsplash.com/photo-1579546929518-9e396f3cc809?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxleHBsb3JlLWZlZWR8MXx8fGVufDB8fHx8&w=1000&q=80"

response = client.responses.create(
    model=MODEL,
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "what's in this image?"},
            {
                "type": "input_image",
                "image_url": image_url,
            },
        ],
    }],
)

print(response.output_text)


This image contains a smooth gradient with a blend of various colors. On the left side, there are shades of pink and red, transitioning through orange and yellow near the center, then shifting to light green, cyan, and finally blue and purple on the bottom right. The colors blend softly into each other, creating a visually appealing gradient effect. There are no distinct objects or shapes present in the image.


---

## 4. Audio Capabilities - Text to Speech

In [24]:
speech_file_path = "speech.mp3"

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="Today is a wonderful day to build something people love!",
    instructions="Speak in a cheerful and positive tone.",
) as response:
    response.stream_to_file(speech_file_path)

---

## 4.1 Audio Capabilities - Speech to Text

In [25]:
audio_file= open("speech.mp3", "rb")

transcription = client.audio.transcriptions.create(
    model="gpt-4o-transcribe", 
    file=audio_file
)

print(transcription.text)

Today's a wonderful day to build something people love!


---

## 5. Function Calling

In [26]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="gpt-4.1-mini",
    input=input_messages,
    tools=tools,
)

In [31]:
# Extract the tool call and arguments
tool_call = response.output[0]
args = json.loads(tool_call.arguments)
# Call the function
result = get_weather(args["latitude"], args["longitude"])

In [32]:
# Append the tool call and result to the input messages
input_messages.append(tool_call) # append model's function call message
input_messages.append({ # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="gpt-4.1-mini",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

The current temperature in Paris is 16 degrees Celsius. If you want more detailed information about the weather, such as conditions (sunny, rainy, etc.), please let me know!


---

## 6. Reasoning Models

In [33]:
# response_o1 = client.responses.create(
#     model="o1",
#     input="Design an algorithm to find the shortest path in a graph."
# )

# print("o1 Response:")
# print(response_o1.output_text)

# Using o4-mini for faster reasoning
response_o4 = client.responses.create(
    model="o4-mini",
    input="Explain how to implement a hash table."
)

print("\no3-mini Response:")
print(response_o4.output_text)


o3-mini Response:
A hash‑table is essentially an array (“table”) plus a hash function that maps each key to an array index.  Collisions—two keys mapping to the same slot—are handled by a “collision‐resolution” strategy.  Here’s a sketch of how you might build one in pseudocode, first with separate chaining and then with open addressing.

1.  Data Structures  
   •  Let N be the number of buckets (initial table size).  
   •  table[0…N–1]: array of bucket‑heads (for chaining), or array of entries (for open addressing).  
   •  Each “entry” stores (key, value), and—if you’re chaining—a pointer to the next entry in that bucket.

2.  A Good Hash Function h(key) → [0…N–1]  
   •  For integers: h(k) = (a·k + b) mod p mod N, with p prime, a≠0.  
   •  For strings: treat as base‑B number, then reduce mod N (or use built‑in string hash).  
   •  Aim for uniform spread.

3.  Separate Chaining  
   – table[i] points to the head of a linked list of all entries whose keys hash to i.  
   a) init: 

In [7]:
## With reasoning models, you can view the reasoning process and the steps taken to arrive at the answer.
print("This is the reasoning configuration for o4-mini:", response_o4.reasoning)

This is the reasoning configuration for o3-mini: Reasoning(effort='medium', generate_summary=None, summary=None)


## 7. Embeddings

In [ ]:
import numpy as np

# First, let's create embeddings for a set of words
words = [
    "king", "queen", "man", "woman", 
    "apple", "banana", "orange", "pear",
    "castle", "throne"
]

# Get embeddings for all words
response = client.embeddings.create(
    model="text-embedding-3-large",
    input=words,
    encoding_format="float"
)

# Extract the embeddings
embeddings = [data.embedding for data in response.data]

print(f"Embedding dimension: {len(embeddings[0])}")
print(f"Number of embeddings: {len(embeddings)}")

# Function to compute dot product between two vectors
def dot_product(vec1, vec2):
    return np.dot(vec1, vec2)

# Compute similarity matrix (dot products between all pairs)
similarity_matrix = np.zeros((len(words), len(words)))
for i in range(len(words)):
    for j in range(len(words)):
        similarity_matrix[i][j] = dot_product(embeddings[i], embeddings[j])

# Print similarity matrix with labels
print("\nSimilarity Matrix (Dot Products):")
print("          " + " ".join(f"{word:<8}" for word in words))
for i, word in enumerate(words):
    row_values = " ".join(f"{similarity_matrix[i][j]:.4f}  " for j in range(len(words)))
    print(f"{word:<10} {row_values}")

# Check specific relationships
king_queen_similarity = dot_product(embeddings[0], embeddings[1])
apple_banana_similarity = dot_product(embeddings[4], embeddings[5])

print("\nSpecific relationships:")
print(f"Similarity between 'king' and 'queen': {king_queen_similarity:.4f}")
print(f"Similarity between 'apple' and 'banana': {apple_banana_similarity:.4f}")
print(f"Similarity between 'king' and 'apple': {dot_product(embeddings[0], embeddings[4]):.4f}")

# We expect king/queen to be closer to each other than king/apple
# And apple/banana to be closer to each other than queen/banana

Embedding dimension: 3072
Number of embeddings: 10

Similarity Matrix (Dot Products):
          king     queen    man      woman    apple    banana   orange   pear     castle   throne  
king       1.0000   0.5524   0.4120   0.2923   0.3221   0.3272   0.2868   0.2761   0.3563   0.3967  
queen      0.5524   1.0000   0.3072   0.4132   0.3145   0.3191   0.2983   0.2996   0.2969   0.3354  
man        0.4120   0.3072   1.0000   0.5713   0.3098   0.3495   0.2972   0.2695   0.2998   0.2650  
woman      0.2923   0.4132   0.5713   1.0000   0.3199   0.2937   0.2784   0.2533   0.2449   0.2491  
apple      0.3221   0.3145   0.3098   0.3199   1.0000   0.4619   0.4588   0.4391   0.3002   0.2340  
banana     0.3272   0.3191   0.3495   0.2937   0.4619   1.0000   0.4579   0.3636   0.2777   0.2075  
orange     0.2868   0.2983   0.2972   0.2784   0.4588   0.4579   1.0000   0.3822   0.2848   0.2174  
pear       0.2761   0.2996   0.2695   0.2533   0.4391   0.3636   0.3822   1.0000   0.2788   0.1929  
castle